<a href="https://colab.research.google.com/github/Naomie25/DI-Bootcamp/blob/main/Week7_Day5_MiniProject_ExerciceXP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install llama_index llama-index-llms-huggingface llama-index-llms-huggingface-api llama-index-embeddings-huggingface vllm


In [4]:
!pip show llama_index


Name: llama-index
Version: 0.12.48
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: 
Author-email: Jerry Liu <jerry@llamaindex.ai>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-question-gen-openai, llama-index-readers-file, llama-index-readers-llama-parse, nltk
Required-by: 


In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


In [10]:
!mkdir -p papers
!wget -O papers/cookbook_ssl.pdf https://arxiv.org/pdf/2304.12210.pdf
!wget -O papers/ssl_survey.pdf https://arxiv.org/pdf/2006.08218.pdf


--2025-07-14 11:38:22--  https://arxiv.org/pdf/2304.12210.pdf
Resolving arxiv.org (arxiv.org)... 151.101.3.42, 151.101.131.42, 151.101.195.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/2304.12210 [following]
--2025-07-14 11:38:22--  http://arxiv.org/pdf/2304.12210
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877344 (1.8M) [application/pdf]
Saving to: ‘papers/cookbook_ssl.pdf’

papers/cookbook_ssl 100%[===================>]   1.79M  --.-KB/s    in 0.07s   

2025-07-14 11:38:23 (24.3 MB/s) - ‘papers/cookbook_ssl.pdf’ saved [1877344/1877344]

--2025-07-14 11:38:23--  https://arxiv.org/pdf/2006.08218.pdf
Resolving arxiv.org (arxiv.org)... 151.101.3.42, 151.101.131.42, 151.101.195.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:443... connected.
HTTP request sent, awaiting response..

In [13]:
llm = HuggingFaceLLM(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    context_window=2048,             # Reasonable for this model size
    max_new_tokens=512,              # Adjust depending on your task
    device_map="auto"                # Automatically chooses CPU/GPU
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [14]:
embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
Settings.llm = llm
Settings.embed_model = embed_model


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
from llama_index.core import SimpleDirectoryReader

# Load PDF documents
documents = SimpleDirectoryReader('papers/').load_data()


In [16]:
from llama_index.core import VectorStoreIndex

# Build the index using global Settings (TinyLlama + Embeddings)
index = VectorStoreIndex.from_documents(documents)

In [17]:
# Persist index to disk
index.storage_context.persist(persist_dir="storage")

In [19]:
query_engine = index.as_query_engine()

In [20]:
response = query_engine.query("What are the main methods discussed in self-supervised learning?")
print(response)

1. Pre-trained Language Models (PTM), Generative Adversarial Networks (GAN), autoencoders, and their extensions. 2. Self-supervised learning’s traditional deﬁnition and LeCun’s deﬁnition, and the “other part” could be incomplete, trans-formed, distorted, or corrupted. 3. Categorization of Self-supervised learning (SSL): Generative, Contrastive, and Generative-Contrastive (Adversarial). 4. Open problems, future directions, and our conclusions.
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
in this big data era,
high-quality data with human labeling could be costly. For
example, Scale.ai 3, a data labeling company, charges $6.4
per image for the image segmentation labeling. An image
segmentation dataset containing 10k+ high-quality samples
could cost up to a million-dollar.
The most crucial point for self-supervised learning’s
success is that it ﬁgures out a way to leverage the tremendous
amounts of unlabeled data that be